In [ ]:
import torch
import torch.nn as nn
from oct_library import OCTProcessing
model_path = '../logs/2022-11-09_19_35_23/checkpoints/model.pth'
oc_file = '../dataset/vol/hc04_spectralis_macula_v1_s1_R.vol'
model = torch.load(model_path, map_location='cpu')
# model = model.half()


In [ ]:
import numpy as np
oct_process = OCTProcessing(oct_file=oc_file, torchmodel=model, half=False, device='cpu') # 125, 36, 10, 68, 15
oct_process.fovea_forward(imgh=256, imgw=256)
# tempCPU = []
# tempGPU = []
# pwCPU = []
# pwGPU = []
# fps = []

for i in range(100):
    oct_process.fovea_forward(imgh=256, imgw=256)
    print(oct_process.ms)
#     fps.append(oct_process.FPS)
#     tempCPU.append(temp_cpu)

# print('FPS', np.array(fps).mean(), np.array(fps).std())
# print('temp CPU', np.array(tempCPU).mean(), np.array(tempCPU).std())
# print('temp GPU',np.array(tempGPU).mean(), np.array(tempGPU).std())
# print('Power CPU',np.array(pwCPU).mean(), np.array(pwCPU).std())
# print('Power GPU',np.array(pwGPU).mean(), np.array(pwGPU).std())

In [ ]:
import torch.onnx
model = torch.load('/media/jetson/CA33-10E2/256.pth')
model.eval()
x = torch.randn(1, 1, 256, 256, requires_grad=True).cuda()

torch.onnx.export(model, x, "unet256_1.onnx", verbose=False)
                    

In [ ]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
f = open("unet256_engine.trt", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING)) 

engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

In [1]:
import torch
from torch2trt import torch2trt
model = torch.load('/media/jetson/CA33-10E2/256.pth')
# create example data
x = torch.ones((1, 1, 256, 256)).cuda()

# convert to TensorRT feeding sample data as input
model_trt = torch2trt(model, [x])

torch.save(model_trt.state_dict(), 'unet256_trt.pth')


/usr/local/lib/python3.6/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch2trt import TRTModule

model_trt = TRTModule()

model_trt.load_state_dict(torch.load('unet256_trt.pth'))

<All keys matched successfully>

: 

In [ ]:
oct_process.plot_slo_fovea()


In [ ]:
oct_process.plot_overlay_oct_segmentation()
oct_process.plot_segmentation_full()
oct_process.get_individual_layers_segmentation(layer=[False, True, True, False, True, True, False, False, True])
oct_process.plot_selected_layers()